In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize

In [ ]:
#testing assets
assets = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'SPY', 'LMT', 'GLD', 'PLTR', 'TSLA', 'NVDA']
data = yf.download(assets, start='2024-01-01')["Adj Close"]
#getting log returns
returns = np.log(data).diff().dropna()

In [ ]:
rf = 0.037

def portfolio_metrics(weights, returns):
    mu = returns.mean() * 252
    cov = returns.cov() * 252

    exp_ret = weights @ mu
    vol = np.sqrt(weights @ cov @ weights)

    sharpe = (exp_ret - rf) / vol
    return exp_ret, vol, sharpe

def maximize_sharpe(weights, returns):
    #means that I get the third element of the return tuple in portfolio metrics (sharpe)
    return -portfolio_metrics(weights, returns)[2]


In [ ]:
def weight_sum_constraint(weights):
    #makes sure weights sum to 1 e.g. all capital is allocated
    return sum(weights) - 1

bounds = [(0.03, 0.33) for _ in range(len(assets))]
constraints = {"type" : "eq", "fun": weight_sum_constraint}

default_weights = np.full(len(assets), 1/len(assets))
result = minimize(maximize_sharpe, default_weights, args=(returns,), method="SLSQP", bounds=bounds, constraints=constraints)

#have to do .x because the result is just an object and the .x is giving the optimized weights back
optimized_weights = result.x
pd.Series(optimized_weights, index=assets)
#this now gives the optimal sharpe bc the weights have changed
portfolio_metrics(optimized_weights, returns)